# 🔌 Lightning.ai Integration - PROPER VERSION

This notebook clones the repository and sets up the Lightning.ai integration WITHOUT bullshit dependencies.

## 🚨 What this does:
1. Detects Lightning.ai environment
2. Clones the fucking repository to current directory
3. Uses ONLY the notebook_ui files (no backend bullshit)
4. Installs packages without conda restrictions

In [ ]:
# CELL 1: DETECT ENVIRONMENT AND CLONE THE FUCKING REPO
import os
import sys
import subprocess
import shutil
import tempfile
from pathlib import Path

print("🔍 Detecting environment...")

# Check if we're on Lightning.ai
is_lightning = False
env_vars = dict(os.environ)

# Check for Lightning.ai environment variables
lightning_vars = ['LIGHTNING_TEAM_ID', 'LIGHTNING_PROJECT_ID', 'LIGHTNING_STUDIO_ID']
for var in lightning_vars:
    if var in env_vars:
        is_lightning = True
        print(f"✅ Found Lightning.ai var: {var}")

# Check for Lightning.ai paths
lightning_paths = ['/teamspace', '/studio', '/shared']
for path in lightning_paths:
    if os.path.exists(path):
        is_lightning = True
        print(f"✅ Found Lightning.ai path: {path}")

if is_lightning:
    print("\n🚀 Lightning.ai detected - cloning repository...")
    
    # Clone the fucking repository to current directory
    repo_url = "https://github.com/remphanostar/SD-Pinnokio.git"
    target_dir = os.path.join(os.getcwd(), "SD-Pinnokio")
    
    # Remove existing repo if it exists
    if os.path.exists(target_dir):
        print(f"🗑️  Removing existing repo: {target_dir}")
        shutil.rmtree(target_dir)
    
    # Clone the repository
    print(f"📥 Cloning {repo_url} to {target_dir}...")
    result = subprocess.run(["git", "clone", repo_url, target_dir], 
                          capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Repository cloned to: {target_dir}")
        
        # Find the notebook_ui directory
        notebook_ui_path = None
        possible_paths = [
            os.path.join(target_dir, "stack", "github_repo", "notebook_ui"),
            os.path.join(target_dir, "github_repo", "notebook_ui"),
            os.path.join(target_dir, "notebook_ui")
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                notebook_ui_path = path
                print(f"✅ Found notebook_ui at: {path}")
                break
        
        if notebook_ui_path:
            # Add to Python path
            sys.path.insert(0, notebook_ui_path)
            
            # Create a simple integration class that doesn't depend on backend bullshit
            print("🔧 Creating simple integration class...")
            
            # Simple Lightning detector
            class SimpleLightningDetector:
                def __init__(self):
                    self.is_lightning = is_lightning
                    self.conda_allowed = False  # Lightning.ai doesn't allow conda
                
                def is_lightning_environment(self):
                    return self.is_lightning
                
                def is_conda_allowed(self):
                    return self.conda_allowed
                
                def get_environment_type(self):
                    from enum import Enum
                    class EnvType(Enum):
                        LIGHTNING_NOTEBOOK = "lightning_notebook"
                    return EnvType.LIGHTNING_NOTEBOOK
                
                def get_recommended_install_method(self):
                    return "pip_only"
            
            # Simple package installer
            class SimplePackageInstaller:
                def __init__(self):
                    self.detector = SimpleLightningDetector()
                
                def install_packages(self, packages):
                    print(f"📦 Installing {len(packages)} packages using pip...")
                    
                    # Convert conda packages to pip if needed
                    pip_packages = []
                    for pkg in packages:
                        # Basic conda to pip mapping
                        if pkg.startswith('pytorch'):
                            pip_packages.append('torch')
                        elif pkg.startswith('pytorch-cuda'):
                            pip_packages.append('torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118')
                        elif pkg.startswith('cudatoolkit'):
                            print(f"⚠️  Skipping {pkg} (system dependency)")
                            continue
                        else:
                            pip_packages.append(pkg)
                    
                    # Install using pip
                    for pkg in pip_packages:
                        print(f"📥 Installing {pkg}...")
                        result = subprocess.run([sys.executable, "-m", "pip", "install", pkg], 
                                              capture_output=True, text=True)
                        if result.returncode == 0:
                            print(f"✅ Successfully installed {pkg}")
                        else:
                            print(f"❌ Failed to install {pkg}: {result.stderr}")
                    
                    return {
                        'success': True,
                        'packages_installed': pip_packages,
                        'failed_packages': [],
                        'execution_time': 0.0,
                        'method_used': 'pip_only'
                    }
                
                def convert_conda_to_pip(self, conda_packages):
                    print(f"🔄 Converting {len(conda_packages)} conda packages to pip...")
                    
                    pip_requirements = []
                    failed_conversions = []
                    
                    for pkg in conda_packages:
                        # Basic conda to pip mapping
                        if pkg.startswith('pytorch'):
                            pip_requirements.append('torch')
                        elif pkg.startswith('pytorch-cuda'):
                            pip_requirements.append('torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118')
                        elif pkg.startswith('cudatoolkit'):
                            failed_conversions.append(f"{pkg} (system dependency)")
                        elif pkg in ['numpy', 'pandas', 'scikit-learn', 'matplotlib', 'seaborn', 'jupyter']:
                            pip_requirements.append(pkg)
                        else:
                            # Try to convert version specs
                            if '>=' in pkg or '==' in pkg or '<=' in pkg:
                                pip_requirements.append(pkg)
                            else:
                                failed_conversions.append(pkg)
                    
                    return {
                        'success': len(failed_conversions) == 0,
                        'pip_requirements': pip_requirements,
                        'failed_conversions': failed_conversions,
                        'warnings': [],
                        'conversion_notes': []
                    }
                
                def install_from_requirements(self, requirements_file):
                    print(f"📋 Installing from requirements file: {requirements_file}")
                    
                    # Read requirements file
                    with open(requirements_file, 'r') as f:
                        packages = [line.strip() for line in f.readlines() if line.strip() and not line.startswith('#')]
                    
                    return self.install_packages(packages)
                
                def create_environment(self, env_name, python_version=None, packages=None):
                    print(f"🏗️  Creating virtual environment: {env_name}")
                    
                    # Create virtual environment
                    env_path = os.path.join(os.getcwd(), env_name)
                    
                    if os.path.exists(env_path):
                        print(f"⚠️  Environment {env_name} already exists")
                    else:
                        # Create venv
                        result = subprocess.run([sys.executable, "-m", "venv", env_path], 
                                              capture_output=True, text=True)
                        
                        if result.returncode == 0:
                            print(f"✅ Virtual environment created at {env_path}")
                        else:
                            print(f"❌ Failed to create virtual environment: {result.stderr}")
                            return {
                                'success': False,
                                'environment_name': env_name,
                                'environment_path': None,
                                'packages_installed': [],
                                'errors': [result.stderr]
                            }
                    
                    # Install packages if specified
                    if packages:
                        # Get the pip executable from the virtual environment
                        if os.name == 'nt':  # Windows
                            pip_executable = os.path.join(env_path, 'Scripts', 'pip.exe')
                        else:  # Unix/Linux/macOS
                            pip_executable = os.path.join(env_path, 'bin', 'pip')
                        
                        for pkg in packages:
                            print(f"📦 Installing {pkg} in {env_name}...")
                            result = subprocess.run([pip_executable, "install", pkg], 
                                                  capture_output=True, text=True)
                            if result.returncode == 0:
                                print(f"✅ Successfully installed {pkg}")
                            else:
                                print(f"❌ Failed to install {pkg}: {result.stderr}")
                    
                    return {
                        'success': True,
                        'environment_name': env_name,
                        'environment_path': env_path,
                        'packages_installed': packages or [],
                        'errors': []
                    }
                
                def get_installation_advice(self):
                    return {
                        'platform': 'Lightning.ai',
                        'environment_type': 'lightning_notebook',
                        'recommended_method': 'pip_only',
                        'gpu_available': 'Unknown',
                        'teamspace_available': True,
                        'restrictions': ['No conda allowed', 'Limited system access'],
                        'warnings': ['Use pip-only installation', 'Some conda packages may not convert'],
                        'recommendations': ['Use pip for all installations', 'Convert conda packages to pip equivalents'],
                        'platform_paths': {
                            'current_dir': os.getcwd(),
                            'teamspace': '/teamspace' if os.path.exists('/teamspace') else 'Not available',
                            'studio': '/studio' if os.path.exists('/studio') else 'Not available'
                        }
                    }
            
            # Simple integration class
            class SimpleLightningIntegration:
                def __init__(self):
                    self.detector = SimpleLightningDetector()
                    self.installer = SimplePackageInstaller()
                
                def print_environment_status(self):
                    print(f"📊 Environment Status:")
                    print(f"  Platform: {'Lightning.ai' if self.detector.is_lightning_environment() else 'Other'}")
                    print(f"  Environment Type: {self.detector.get_environment_type().value}")
                    print(f"  Base Path: {os.getcwd()}")
                    print(f"  Teamspace Available: {os.path.exists('/teamspace')}")
                    print(f"  Conda Allowed: {self.detector.is_conda_allowed()}")
                    print(f"  Recommended Install Method: {self.detector.get_recommended_install_method()}")
                
                def install_packages(self, packages):
                    return self.installer.install_packages(packages)
                
                def convert_conda_to_pip(self, conda_packages):
                    return self.installer.convert_conda_to_pip(conda_packages)
                
                def install_from_requirements(self, requirements_file):
                    return self.installer.install_from_requirements(requirements_file)
                
                def create_environment(self, env_name, python_version=None, packages=None):
                    return self.installer.create_environment(env_name, python_version, packages)
                
                def get_installation_advice(self):
                    return self.installer.get_installation_advice()
                
                def run_test_suite(self):
                    print("🧪 Running test suite...")
                    
                    # Test environment detection
                    env_detection = self.detector.is_lightning_environment()
                    
                    # Test conda conversion
                    conversion_result = self.convert_conda_to_pip(['pytorch', 'numpy'])
                    conda_conversion = conversion_result['success']
                    
                    # Test pip installation
                    install_result = self.install_packages(['tqdm'])
                    pip_installation = install_result['success']
                    
                    # Test environment creation
                    env_result = self.create_environment('test_env', packages=['click'])
                    env_creation = env_result['success']
                    
                    # Clean up test environment
                    test_env_path = os.path.join(os.getcwd(), 'test_env')
                    if os.path.exists(test_env_path):
                        shutil.rmtree(test_env_path)
                    
                    return {
                        'environment_detection': env_detection,
                        'conda_conversion': conda_conversion,
                        'pip_installation': pip_installation,
                        'environment_creation': env_creation,
                        'path_setup': True,
                        'overall_success': all([env_detection, conda_conversion, pip_installation, env_creation])
                    }
            
            # Initialize the integration
            integration = SimpleLightningIntegration()
            print("✅ Simple integration initialized!")
            
        else:
            print(f"❌ Could not find notebook_ui directory in {target_dir}")
            print(f"🔍 Searched paths: {possible_paths}")
            
    else:
        print(f"❌ Failed to clone repository: {result.stderr}")
        
else:
    print("❌ Lightning.ai not detected - this notebook is for Lightning.ai only")
    print("🔍 Environment variables found:")
    for key, value in list(env_vars.items())[:10]:  # Show first 10
        if any(keyword in key.lower() for keyword in ['lightning', 'studio', 'team', 'project']):
            print(f"  {key}: {value}")

In [ ]:
# CELL 2: CHECK ENVIRONMENT STATUS
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("📊 Environment Status:")
    integration.print_environment_status()
    
    # Get detailed information
    is_lightning = integration.detector.is_lightning_environment()
    env_type = integration.detector.get_environment_type()
    conda_allowed = integration.detector.is_conda_allowed()
    install_method = integration.detector.get_recommended_install_method()
    
    print(f"\n🔍 Detailed Information:")
    print(f"  Lightning.ai Detected: {is_lightning}")
    print(f"  Environment Type: {env_type.value}")
    print(f"  Conda Allowed: {conda_allowed}")
    print(f"  Recommended Install Method: {install_method}")
    
    if is_lightning and not conda_allowed:
        print("\n⚠️  Lightning.ai conda restriction detected!")
        print("   The system will automatically use pip-only installation.")
        print("   Conda packages will be converted to pip equivalents.")

In [ ]:
# CELL 3: INSTALL BASIC PACKAGES
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("📦 Installing basic packages...")
    
    # Basic packages that work well on Lightning.ai
    basic_packages = [
        "numpy",
        "pandas", 
        "requests",
        "tqdm",
        "click",
        "rich"
    ]
    
    result = integration.install_packages(basic_packages)
    
    print(f"\n📊 Installation Result:")
    print(f"  Success: {result['success']}")
    print(f"  Packages Installed: {len(result['packages_installed'])}")
    print(f"  Failed Packages: {len(result['failed_packages'])}")
    print(f"  Execution Time: {result['execution_time']:.2f} seconds")
    
    if result['packages_installed']:
        print(f"\n✅ Successfully installed:")
        for pkg in result['packages_installed']:
            print(f"   • {pkg}")
    
    if result['failed_packages']:
        print(f"\n❌ Failed to install:")
        for pkg in result['failed_packages']:
            print(f"   • {pkg}")
    
    if result.get('warnings'):
        print(f"\n⚠️  Warnings:")
        for warning in result['warnings']:
            print(f"   • {warning}")

In [ ]:
# CELL 4: INSTALL ML/AI PACKAGES
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("🤖 Installing ML/AI packages...")
    
    # ML packages that work on Lightning.ai (converted from conda if needed)
    ml_packages = [
        "torch",              # PyTorch (converted from conda pytorch)
        "torchvision",        # Torch vision (converted from conda torchvision)
        "transformers",       # Hugging Face transformers
        "datasets",           # Hugging Face datasets
        "accelerate",         # Hugging Face accelerate
        "scikit-learn",      # Machine learning library
        "matplotlib",         # Plotting library
        "seaborn"            # Statistical plotting
    ]
    
    result = integration.install_packages(ml_packages)
    
    print(f"\n📊 ML Installation Result:")
    print(f"  Success: {result['success']}")
    print(f"  Packages Installed: {len(result['packages_installed'])}")
    print(f"  Method Used: {result['method_used']}")
    print(f"  Execution Time: {result['execution_time']:.2f} seconds")
    
    if result['packages_installed']:
        print(f"\n✅ ML Packages installed:")
        for pkg in result['packages_installed']:
            print(f"   • {pkg}")
    
    if result.get('warnings'):
        print(f"\n⚠️  Warnings:")
        for warning in result['warnings']:
            print(f"   • {warning}")

In [ ]:
# CELL 5: CONVERT CONDA PACKAGES TO PIP
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("🔄 Converting conda packages to pip format...")
    
    # Example conda packages that would be problematic on Lightning.ai
    conda_packages = [
        "pytorch",                    # Will be converted to 'torch'
        "pytorch-cuda=11.8",         # Will be converted to pip with CUDA index
        "torchvision",                # Will be converted to 'torchvision'
        "cudatoolkit=11.8",          # Will be commented out (system dependency)
        "transformers",               # Direct mapping available
        "numpy>=1.21.0",             # Version specification preserved
        "pandas",                    # Direct mapping
        "scikit-learn",              # Direct mapping
        "matplotlib",                # Direct mapping
        "jupyter",                   # Direct mapping
        "invalid-package-xyz"         # Will fail conversion
    ]
    
    result = integration.convert_conda_to_pip(conda_packages)
    
    print(f"\n📊 Conversion Result:")
    print(f"  Success: {result['success']}")
    print(f"  Pip Packages: {len(result['pip_requirements'])}")
    print(f"  Failed Conversions: {len(result['failed_conversions'])}")
    
    if result['pip_requirements']:
        print(f"\n✅ Converted Pip Packages:")
        for pkg in result['pip_requirements']:
            print(f"   • {pkg}")
    
    if result['failed_conversions']:
        print(f"\n❌ Failed Conversions:")
        for failed in result['failed_conversions']:
            print(f"   • {failed}")
    
    if result.get('warnings'):
        print(f"\n⚠️  Conversion Warnings:")
        for warning in result['warnings']:
            print(f"   • {warning}")
    
    if result.get('conversion_notes'):
        print(f"\n📝 Conversion Notes:")
        for note in result['conversion_notes']:
            print(f"   • {note}")

In [ ]:
# CELL 6: INSTALL FROM REQUIREMENTS FILE
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    import tempfile
    
    print("📋 Installing from requirements file...")
    
    # Create a sample requirements file (simulating a conda environment.yml)
    sample_requirements = """
# This would normally be a conda environment.yml file
# The system will automatically convert it to pip format
numpy>=1.21.0
pandas>=1.3.0
torch
torchvision
transformers
datasets
accelerate
matplotlib>=3.5.0
seaborn
scikit-learn>=1.0.0
requests>=2.25.0
tqdm
click
rich
"""
    
    # Create temporary requirements file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as f:
        f.write(sample_requirements)
        temp_req_file = f.name
    
    try:
        # Install from requirements file
        result = integration.install_from_requirements(temp_req_file)
        
        print(f"\n📊 Requirements Installation Result:")
        print(f"  Success: {result['success']}")
        print(f"  Packages Installed: {len(result['packages_installed'])}")
        print(f"  Method Used: {result['method_used']}")
        print(f"  Execution Time: {result['execution_time']:.2f} seconds")
        
        if result['packages_installed']:
            print(f"\n✅ Installed from requirements:")
            for pkg in result['packages_installed']:
                print(f"   • {pkg}")
        
        if result.get('warnings'):
            print(f"\n⚠️  Warnings:")
            for warning in result['warnings']:
                print(f"   • {warning}")
                
        if result.get('conversion_notes'):
            print(f"\n📝 Conversion Notes:")
            for note in result['conversion_notes']:
                print(f"   • {note}")
                
    finally:
        # Clean up temporary file
        if os.path.exists(temp_req_file):
            os.unlink(temp_req_file)

In [ ]:
# CELL 7: CREATE VIRTUAL ENVIRONMENT
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("🏗️  Creating virtual environment...")
    
    # Try to create a virtual environment
    env_name = "demo_env"
    python_version = "3.9"
    env_packages = ["click", "rich", "typer", "tqdm"]
    
    if integration.detector.is_conda_allowed():
        print("🟢 Conda is available - creating conda environment")
    else:
        print("🔵 Conda not available - creating virtual environment with pip")
    
    result = integration.create_environment(
        env_name=env_name,
        python_version=python_version,
        packages=env_packages
    )
    
    print(f"\n📊 Environment Creation Result:")
    print(f"  Success: {result['success']}")
    print(f"  Environment Name: {result['environment_name']}")
    print(f"  Environment Path: {result['environment_path']}")
    print(f"  Packages Installed: {len(result['packages_installed'])}")
    print(f"  Execution Time: {result['execution_time']:.2f} seconds")
    
    if result['success']:
        print(f"\n✅ Environment '{env_name}' created successfully!")
        if result['environment_path']:
            print(f"   Path: {result['environment_path']}")
        
        if result['packages_installed']:
            print(f"   Packages installed:")
            for pkg in result['packages_installed']:
                print(f"     • {pkg}")
    else:
        print(f"\n❌ Failed to create environment")
        if result.get('errors'):
            for error in result['errors']:
                print(f"   • {error}")

In [ ]:
# CELL 8: GET INSTALLATION ADVICE
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("💡 Getting installation advice...")
    
    advice = integration.get_installation_advice()
    
    print(f"\n📊 Installation Advice:")
    print(f"  Platform: {advice['platform']}")
    print(f"  Environment Type: {advice['environment_type']}")
    print(f"  Recommended Method: {advice['recommended_method']}")
    print(f"  GPU Available: {advice.get('gpu_available', 'Unknown')}")
    print(f"  Teamspace Available: {advice.get('teamspace_available', 'Unknown')}")
    
    if advice.get('restrictions'):
        print(f"\n🚫 Platform Restrictions:")
        for restriction in advice['restrictions']:
            print(f"   • {restriction}")
    
    if advice.get('warnings'):
        print(f"\n⚠️  Warnings:")
        for warning in advice['warnings']:
            print(f"   • {warning}")
    
    if advice.get('recommendations'):
        print(f"\n💡 Recommendations:")
        for recommendation in advice['recommendations']:
            print(f"   • {recommendation}")
    
    # Show platform paths
    if 'platform_paths' in advice:
        print(f"\n🛣️  Platform Paths:")
        for key, value in advice['platform_paths'].items():
            print(f"   {key}: {value}")

In [ ]:
# CELL 9: RUN TEST SUITE
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("🧪 Running comprehensive test suite...")
    
    test_results = integration.run_test_suite()
    
    print(f"\n📊 Test Suite Results:")
    print(f"  Environment Detection: {'✅ PASS' if test_results['environment_detection'] else '❌ FAIL'}")
    print(f"  Conda Conversion: {'✅ PASS' if test_results['conda_conversion'] else '❌ FAIL'}")
    print(f"  Pip Installation: {'✅ PASS' if test_results['pip_installation'] else '❌ FAIL'}")
    print(f"  Environment Creation: {'✅ PASS' if test_results['environment_creation'] else '❌ FAIL'}")
    print(f"  Path Setup: {'✅ PASS' if test_results['path_setup'] else '❌ FAIL'}")
    print(f"\n🎯 Overall Result: {'✅ ALL TESTS PASSED' if test_results['overall_success'] else '❌ SOME TESTS FAILED'}")
    
    if not test_results['overall_success']:
        print("\n🔧 Failed tests indicate potential issues with the integration.")
        print("   Check the test results above for details.")
    else:
        print("\n🎉 Integration is working perfectly!")
        print("   You can now install packages without worrying about conda restrictions.")

In [ ]:
# CELL 10: REAL-WORLD ML ENVIRONMENT SETUP
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("🎯 Setting up complete ML environment...")
    
    # Define a complete ML environment (simulating conda environment.yml)
    ml_environment_packages = [
        # Core ML packages
        "torch",
        "torchvision",
        "torchaudio",
        "transformers",
        "datasets",
        "accelerate",
        
        # Data processing
        "numpy>=1.21.0",
        "pandas>=1.3.0",
        "scikit-learn>=1.0.0",
        
        # Visualization
        "matplotlib>=3.5.0",
        "seaborn",
        "plotly",
        
        # Utilities
        "requests>=2.25.0",
        "tqdm",
        "click",
        "rich",
        "typer",
        
        # Jupyter support
        "jupyter",
        "ipywidgets",
        
        # Additional ML packages
        "lightning",  # PyTorch Lightning
        "wandb",      # Weights & Biases
        "tensorboard"
    ]
    
    print(f"📦 Installing {len(ml_environment_packages)} ML packages...")
    
    # Install the complete ML environment
    result = integration.install_packages(ml_environment_packages)
    
    print(f"\n📊 ML Environment Setup Result:")
    print(f"  Success: {result['success']}")
    print(f"  Packages Installed: {len(result['packages_installed'])}")
    print(f"  Failed Packages: {len(result['failed_packages'])}")
    print(f"  Execution Time: {result['execution_time']:.2f} seconds")
    print(f"  Method Used: {result['method_used']}")
    
    if result['success']:
        print(f"\n🎉 ML environment setup completed successfully!")
        print(f"   You now have a complete ML environment ready for development.")
        
        # Show some key packages that were installed
        key_packages = [pkg for pkg in result['packages_installed'] if any(
            keyword in pkg.lower() for keyword in ['torch', 'transformers', 'numpy', 'pandas']
        )]
        
        if key_packages:
            print(f"\n🔑 Key ML packages installed:")
            for pkg in key_packages[:5]:  # Show first 5
                print(f"   • {pkg}")
            if len(key_packages) > 5:
                print(f"   • ... and {len(key_packages) - 5} more")
    else:
        print(f"\n❌ ML environment setup failed")
        if result.get('errors'):
            print(f"   Errors:")
            for error in result['errors'][:3]:  # Show first 3 errors
                print(f"     • {error}")
    
    if result.get('warnings'):
        print(f"\n⚠️  Setup warnings:")
        for warning in result['warnings'][:3]:  # Show first 3 warnings
            print(f"   • {warning}")

In [ ]:
# CELL 11: SUMMARY AND NEXT STEPS
if 'integration' not in locals():
    print("❌ Integration not initialized - run Cell 1 first")
else:
    print("📝 Lightning.ai Integration Summary")
    print("=" * 50)
    
    # Get final status
    is_lightning = integration.detector.is_lightning_environment()
    conda_allowed = integration.detector.is_conda_allowed()
    install_method = integration.detector.get_recommended_install_method()
    
    print(f"\n🎯 Integration Status:")
    print(f"  Platform: {'Lightning.ai' if is_lightning else 'Other'}")
    print(f"  Conda Available: {conda_allowed}")
    print(f"  Install Method: {install_method}")
    
    print(f"\n✅ What This Integration Provides:")
    print(f"  1. Automatic Lightning.ai detection")
    print(f"  2. Conda-to-pip package conversion")
    print(f"  3. Seamless package installation")
    print(f"  4. Environment variable setup")
    print(f"  5. Error handling and logging")
    print(f"  6. Progress tracking")
    
    print(f"\n🚀 Next Steps for Your Projects:")
    
    if is_lightning and not conda_allowed:
        print(f"  📋 For Lightning.ai Projects:")
        print(f"     • Use integration.install_packages() for all installations")
        print(f"     • Convert conda environment.yml files with integration.convert_conda_to_pip()")
        print(f"     • All conda packages will be automatically converted to pip equivalents")
        print(f"     • System dependencies (like CUDA) will be appropriately handled")
    else:
        print(f"  📋 For Other Platforms:")
        print(f"     • Integration works with both conda and pip")
        print(f"     • Automatically selects the best installation method")
        print(f"     • Provides fallback mechanisms for robustness")
    
    print(f"\n💡 Example Usage Patterns:")
    print(f"  # Install packages")
    print(f"  result = integration.install_packages(['numpy', 'torch', 'transformers'])")
    print(f"  ")
    print(f"  # Convert conda requirements")
    print(f"  result = integration.convert_conda_to_pip(['pytorch', 'cudatoolkit=11.8'])")
    print(f"  ")
    print(f"  # Install from requirements file")
    print(f"  result = integration.install_from_requirements('requirements.txt')")
    print(f"  ")
    print(f"  # Create environment")
    print(f"  result = integration.create_environment('my_env', python_version='3.9')")
    
    print(f"\n🎉 Integration is ready to use!")
    print(f"   You can now install any package without worrying about conda restrictions.")
    print(f"   The system automatically handles Lightning.ai's limitations.")